## 나이브베이즈 타이타닉 정확도 코드 (반복문으로 확인용!!)

In [3]:
import pandas as pd
# 1단계. csv --> 데이터 프레임으로 변환
df = pd.read_csv('d:\\data\\tat\\train.csv')

# 컬럼이 모두다 출력될 수 있도록 출력할 열의 개수 한도를 늘리기
pd.set_option('display.max_columns', 15)



# 2.1 결측치를 확인하고 제거하거나 치환한다.
# 타이타닉 데이터 프레임의 자료형을 확인한다.
mask1 = (df['Age'] < 10) | (df['Sex'] == 'female')
df['child_women'] = mask1.astype(int)
# 2.2 결측치(NaN)을 확인한다.

# 2.3 deck 컬럼과 Cabin 컬럼을 삭제한다.
rdf = df.drop(['Cabin'], axis =1)


# 2.4 Age(나이) 열에 나이가 없는 모든 행을 삭제한다.
# age 열에 891개의 행중에 177개가 NaN값

rdf = rdf.dropna(subset = ['Age'], how='any', axis = 0)


# 설명
# 데이터가 한개라도 없으면 drop 해라 (how = 'any')
# 모든 데이터가 없으면 drop 해라 (how = 'all')

# 2.5 embark 열의 NaN값을 승선도시중 가장 많이 출현한 값으로 치환하기.

most_freq = rdf['Embarked'].value_counts(dropna=True).idxmax()
rdf['Embarked'].fillna(most_freq, inplace = True)


# 3. 범주형 데이터를 숫자형으로 변환하기

# 3.1 feature selection (분석에 필요한 속성을 선택)
ndf = rdf[['Survived','Pclass','Sex','Age','SibSp','Parch','Embarked', 'child_women']]
print(ndf.head(), '\n')


# 선택된 컬럼중 2개(sex, embarked)가 범주형(문자형)이다.

# 3.2 범주형 데이터를 숫자로 변환(원핫 인코딩)
gender = pd.get_dummies(ndf['Sex'])
ndf = pd.concat([ndf,gender], axis = 1)

onehot_embarked = pd.get_dummies(ndf['Embarked'], prefix = 'town')
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

ndf.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)


# 4단계. 정규화
# survived  pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S
# survived (종속변수 y)
# pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S (독립변수 X)

# 4.1 독립변수와 종속변수(라벨)를 지정한다.
X = ndf[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S', 'child_women']] # 독립변수
y = ndf['Survived'] # 종속변수

# 4.2 독립변수들을 정규화(normalization) 한다.
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)


# 4.2 독립변수들을 정규화(normalization) 한다.
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)


# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다. (반복문까지 추가함!!!!)

u = []
import numpy as np
al = list(np.arange(1,100,10))

for i in range(90,100):
    
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    
    # 설명 : test_size = 0.3 에 의해서 7:3 비율로 훈련과 테스트를 나누고 
    # random_state = 10 에 의해서 나중에 split할 때도 항상 일정하게 split 할 수 있게 한다.

    
    # 6단계. 머신러닝 모델을 생성한다.(knn을 사용)
    for j in al:
        from sklearn.naive_bayes import GaussianNB, BernoulliNB
        
        model = BernoulliNB(alpha= j) # 이걸로 라플라스 값을 지정할 수도 있다.
        model.fit(X_train, y_train)
        
        # 7단계. 테스트 데이터로 예측하기
        
        y_hat = model.predict(X_test)

   
        print('alpha =', j, 'random_state =', i)
        
        # 9단계 정확도 확인
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score( y_test, y_hat)
        print('accuracy는', accuracy)

        u.append(accuracy)
        print('\n')
        


print('최대정확도는 : ', max(u))
print(u.index(max(u)), '번째순서')

   Survived  Pclass     Sex   Age  SibSp  Parch Embarked
0         0       3    male  22.0      1      0        S
1         1       1  female  38.0      1      0        C
2         1       3  female  26.0      0      0        S
3         1       1  female  35.0      1      0        S
4         0       3    male  35.0      0      0        S 

alpha = 1 random_state = 90
accuracy는 0.7581395348837209


alpha = 11 random_state = 90
accuracy는 0.7674418604651163


alpha = 21 random_state = 90
accuracy는 0.7674418604651163


alpha = 31 random_state = 90
accuracy는 0.7674418604651163


alpha = 41 random_state = 90
accuracy는 0.7674418604651163


alpha = 51 random_state = 90
accuracy는 0.7674418604651163


alpha = 61 random_state = 90
accuracy는 0.7674418604651163


alpha = 71 random_state = 90
accuracy는 0.7767441860465116


alpha = 81 random_state = 90
accuracy는 0.7767441860465116


alpha = 91 random_state = 90
accuracy는 0.7767441860465116


alpha = 1 random_state = 91
accuracy는 0.8


alpha = 11 ra

## 반복문 x, 나이브 베이즈 코드 (submission.csv제출용 코드)

In [6]:
import pandas as pd


df = pd.read_csv("d:\\data\\tat\\train.csv")
df2 = pd.read_csv("d:\\data\\tat\\test.csv")

pd.set_option('display.max_columns', 15)


mask1 = (df['Age'] < 10) | (df['Sex'] == 'female')
mask2 = (df2['Age'] < 10) | (df2['Sex'] == 'female')

df['child_women'] = mask1.astype(int)
df2['child_women'] = mask2.astype(int)


print(df.info())
print(df2.info())
print('\n')

print(df.isnull().sum(axis=0), '\n')
print(df2.isnull().sum(axis=0), '\n')

rdf = df.drop(['Cabin'], axis =1)
rdf2 = df2.drop(['Cabin'], axis =1)

print(rdf.columns.values, '\n')
print(rdf2.columns.values, '\n')

most_mean = rdf2['Age'].mean()
rdf = rdf.dropna(subset = ['Age'], how='any', axis = 0)
rdf2['Age'].fillna(most_mean, inplace = True)

print(len(rdf), '\n')
print(rdf2, '\n')



most_freq = rdf['Embarked'].value_counts(dropna=True).idxmax()
most_freq2 = rdf2['Fare'].value_counts(dropna=True).idxmax()
rdf['Embarked'].fillna(most_freq, inplace = True)
rdf2['Fare'].fillna(most_freq2, inplace = True)

print(rdf.isnull().sum(axis=0), '\n')
print(rdf2.isnull().sum(axis=0), '\n')


ndf = rdf[['Survived','Pclass','Sex','Age','SibSp','Parch','Embarked', 'child_women']]
ndf2 = rdf2[['Pclass','Sex','Age','SibSp','Parch','Embarked', 'child_women']]

print(ndf.head(), '\n')
print(ndf2.head(), '\n')

gender = pd.get_dummies(ndf['Sex'])
gender2 = pd.get_dummies(ndf2['Sex'])

ndf = pd.concat([ndf,gender], axis = 1)
ndf2 = pd.concat([ndf2,gender2], axis = 1)
#
onehot_embarked = pd.get_dummies(ndf['Embarked'], prefix = 'town') # 접두사를 정함.
onehot_embarked2 = pd.get_dummies(ndf2['Embarked'], prefix = 'town') # 접두사를 정함.

ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf2 = pd.concat([ndf2, onehot_embarked2], axis = 1)

ndf.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)
ndf2.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)

print(ndf.head())
print('\n')

print(ndf2.head())
print('\n')

X = ndf[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S', 'child_women']] # 독립변수
y = ndf['Survived'] # 종속변수

test = ndf2[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S', 'child_women']]

from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
test = preprocessing.StandardScaler().fit(test).transform(test)

print(X)
print(test)

# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 94)



from sklearn.naive_bayes import GaussianNB, BernoulliNB

# 모형 객체 생성
model = BernoulliNB(alpha=31) # 이걸로 라플라스 값을 지정할 수도 있다.
model.fit(X_train, y_train) # model.fit으로 훈련시킨다.


y_hat = model.predict(X_test)

# Prediction
y_pred = model.predict(test)


# 정확도 : (109 + 65) / (109 + 16 + 25 + 65)
# 정확도 100%는 현실적으로 나오기 힘드니 (예측: 생존 , 실제: 사망 데이터인 25를 0으로 만드는 것을 궁극적으로 추구한다.)

# confusion matrix에 대한 변수 지정 (tn,fp,fn,tp)
from sklearn import metrics
tn, fp, fn, tp = metrics.confusion_matrix( y_test, y_hat ).ravel()
print(tn,fp,fn,tp)
print('\n')


# 9단계 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
print(accuracy)


sample_submission=pd.read_csv('d:\\data\\tat\\gender_submission.csv', index_col=0)


# Submission
submission = pd.DataFrame(data=y_pred,columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('d:\\data\\tat\\submission2.csv', index=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
child_women    891 non-null int32
dtypes: float64(2), int32(1), int64(5), object(5)
memory usage: 87.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 

## 반복문 x, knn 코드 (submission.csv제출용 코드)

In [4]:
import pandas as pd


df = pd.read_csv("d:\\data\\tat\\train.csv")
df2 = pd.read_csv("d:\\data\\tat\\test.csv")

pd.set_option('display.max_columns', 15)


mask1 = (df['Age'] < 10) | (df['Sex'] == 'female')
mask2 = (df2['Age'] < 10) | (df2['Sex'] == 'female')

df['child_women'] = mask1.astype(int)
df2['child_women'] = mask2.astype(int)


print(df.info())
print(df2.info())
print('\n')

print(df.isnull().sum(axis=0), '\n')
print(df2.isnull().sum(axis=0), '\n')

rdf = df.drop(['Cabin'], axis =1)
rdf2 = df2.drop(['Cabin'], axis =1)

print(rdf.columns.values, '\n')
print(rdf2.columns.values, '\n')

most_mean = rdf2['Age'].mean()
rdf = rdf.dropna(subset = ['Age'], how='any', axis = 0)
rdf2['Age'].fillna(most_mean, inplace = True)

print(len(rdf), '\n')
print(rdf2, '\n')



most_freq = rdf['Embarked'].value_counts(dropna=True).idxmax()
most_freq2 = rdf2['Fare'].value_counts(dropna=True).idxmax()
rdf['Embarked'].fillna(most_freq, inplace = True)
rdf2['Fare'].fillna(most_freq2, inplace = True)

print(rdf.isnull().sum(axis=0), '\n')
print(rdf2.isnull().sum(axis=0), '\n')


ndf = rdf[['Survived','Pclass','Sex','Age','SibSp','Parch','Embarked', 'child_women']]
ndf2 = rdf2[['Pclass','Sex','Age','SibSp','Parch','Embarked', 'child_women']]

print(ndf.head(), '\n')
print(ndf2.head(), '\n')

gender = pd.get_dummies(ndf['Sex'])
gender2 = pd.get_dummies(ndf2['Sex'])

ndf = pd.concat([ndf,gender], axis = 1)
ndf2 = pd.concat([ndf2,gender2], axis = 1)
#
onehot_embarked = pd.get_dummies(ndf['Embarked'], prefix = 'town') # 접두사를 정함.
onehot_embarked2 = pd.get_dummies(ndf2['Embarked'], prefix = 'town') # 접두사를 정함.

ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf2 = pd.concat([ndf2, onehot_embarked2], axis = 1)

ndf.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)
ndf2.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)

print(ndf.head())
print('\n')

print(ndf2.head())
print('\n')

X = ndf[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S', 'child_women']] # 독립변수
y = ndf['Survived'] # 종속변수

test = ndf2[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S', 'child_women']]

from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
test = preprocessing.StandardScaler().fit(test).transform(test)


# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 94)



from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5) # n_neighbors = 5 (knn의 k값은 5) 변수
knn.fit(X_train, y_train)


y_hat = knn.predict(X_test)

# Prediction
y_pred = knn.predict(test)


from sklearn import metrics
naive_matrix = metrics.confusion_matrix(y_test,y_hat)

# confusion matrix에 대한 변수 지정 (tn,fp,fn,tp)
from sklearn import metrics
tn, fp, fn, tp = metrics.confusion_matrix( y_test, y_hat ).ravel()



# 9단계 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
print(accuracy)


sample_submission=pd.read_csv('d:\\data\\tat\\gender_submission.csv', index_col=0)

# Submission
submission = pd.DataFrame(data=y_pred,columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('d:\\data\\tat\\submission2.csv', index=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
child_women    891 non-null int32
dtypes: float64(2), int32(1), int64(5), object(5)
memory usage: 87.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 

## 반복문 x, 의사결정트리 코드 (submission.csv제출용 코드)

In [ ]:
import pandas as pd


df = pd.read_csv("d:\\data\\tat\\train.csv")
df2 = pd.read_csv("d:\\data\\tat\\test.csv")

pd.set_option('display.max_columns', 15)


mask1 = (df['Age'] < 10) | (df['Sex'] == 'female')
mask2 = (df2['Age'] < 10) | (df2['Sex'] == 'female')

df['child_women'] = mask1.astype(int)
df2['child_women'] = mask2.astype(int)


print(df.info())
print(df2.info())
print('\n')

print(df.isnull().sum(axis=0), '\n')
print(df2.isnull().sum(axis=0), '\n')

rdf = df.drop(['Cabin'], axis =1)
rdf2 = df2.drop(['Cabin'], axis =1)

print(rdf.columns.values, '\n')
print(rdf2.columns.values, '\n')

most_mean = rdf2['Age'].mean()
rdf = rdf.dropna(subset = ['Age'], how='any', axis = 0)
rdf2['Age'].fillna(most_mean, inplace = True)

print(len(rdf), '\n')
print(rdf2, '\n')



most_freq = rdf['Embarked'].value_counts(dropna=True).idxmax()
most_freq2 = rdf2['Fare'].value_counts(dropna=True).idxmax()
rdf['Embarked'].fillna(most_freq, inplace = True)
rdf2['Fare'].fillna(most_freq2, inplace = True)

print(rdf.isnull().sum(axis=0), '\n')
print(rdf2.isnull().sum(axis=0), '\n')


ndf = rdf[['Survived','Pclass','Sex','Age','SibSp','Parch','Embarked', 'child_women']]
ndf2 = rdf2[['Pclass','Sex','Age','SibSp','Parch','Embarked', 'child_women']]

print(ndf.head(), '\n')
print(ndf2.head(), '\n')

gender = pd.get_dummies(ndf['Sex'])
gender2 = pd.get_dummies(ndf2['Sex'])

ndf = pd.concat([ndf,gender], axis = 1)
ndf2 = pd.concat([ndf2,gender2], axis = 1)
#
onehot_embarked = pd.get_dummies(ndf['Embarked'], prefix = 'town') # 접두사를 정함.
onehot_embarked2 = pd.get_dummies(ndf2['Embarked'], prefix = 'town') # 접두사를 정함.

ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf2 = pd.concat([ndf2, onehot_embarked2], axis = 1)

ndf.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)
ndf2.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)

print(ndf.head())
print('\n')

print(ndf2.head())
print('\n')

X = ndf[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S', 'child_women']] # 독립변수
y = ndf['Survived'] # 종속변수

test = ndf2[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S', 'child_women']]

from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
test = preprocessing.StandardScaler().fit(test).transform(test)

print(X)
print(test)

# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 94)



from sklearn import tree
tree_model = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = 3)
tree_model.fit(X_train, y_train)

# 7단계. 테스트 데이터로 예측하기

y_hat = tree_model.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

# Prediction
y_pred = tree_model.predict(test)


# 정확도 : (109 + 65) / (109 + 16 + 25 + 65)
# 정확도 100%는 현실적으로 나오기 힘드니 (예측: 생존 , 실제: 사망 데이터인 25를 0으로 만드는 것을 궁극적으로 추구한다.)

# confusion matrix에 대한 변수 지정 (tn,fp,fn,tp)
from sklearn import metrics
tn, fp, fn, tp = metrics.confusion_matrix( y_test, y_hat ).ravel()
print(tn,fp,fn,tp)
print('\n')


# 9단계 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
print(accuracy)


sample_submission=pd.read_csv('d:\\data\\tat\\gender_submission.csv', index_col=0)


# Submission
submission = pd.DataFrame(data=y_pred,columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('d:\\data\\tat\\submission2.csv', index=True)

## 반복문 o, 의사결정트리 코드(seaborn 코드)

In [ ]:
# Full code.
import seaborn as sns
import pandas as pd

# 1단계. csv --> 데이터 프레임으로 변환
df = sns.load_dataset('titanic')

# 컬럼이 모두다 출력될 수 있도록 출력할 열의 개수 한도를 늘리기
pd.set_option('display.max_columns', 15)
print(df.head(), '\n')

mask1 = (df['age'] < 10) | (df['sex'] == 'female')
df['child_women'] = mask1.astype(int)

# 2.1 결측치를 확인하고 제거하거나 치환한다.
# 타이타닉 데이터 프레임의 자료형을 확인한다.

print(df.info()) # 자료형 확인하는 방법
print('\n')
# 2.2 결측치(NaN)을 확인한다.
print(df.isnull().sum(axis=0), '\n')

# 설명 : deck 결측치가 많아서 컬럼을 삭제해야함
# embark 와 embark_town이 같은 데이터(중복데이터)여서 컬럼을 하나 삭제해야함.
# embark 컬럼은 삭제한다.

# 2.3 deck 컬럼과 embark_town 컬럼을 삭제한다.
rdf = df.drop(['deck', 'embark_town'], axis =1)
print(rdf.columns.values, '\n')

# 2.4 age(나이) 열에 나이가 없는 모든 행을 삭제한다.
# age 열에 891개의 행중에 177개가 NaN값

rdf = rdf.dropna(subset = ['age'], how='any', axis = 0)
print(len(rdf), '\n')

# 설명
# 데이터가 한개라도 없으면 drop 해라 (how = 'any')
# 모든 데이터가 없으면 drop 해라 (how = 'all')

# 2.5 embark 열의 NaN값을 승선도시중 가장 많이 출현한 값으로 치환하기.

most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace = True)
print(rdf.isnull().sum(axis=0), '\n')

# 3. 범주형 데이터를 숫자형으로 변환하기

# 3.1 feature selection (분석에 필요한 속성을 선택)
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked', 'child_women']]
print(ndf.head(), '\n')
# 선택된 컬럼중 2개(sex, embarked)가 범주형(문자형)이다.

# 3.2 범주형 데이터를 숫자로 변환(원핫 인코딩)
gender = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf,gender], axis = 1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix = 'town') # 접두사를 정함.
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

ndf.drop(['sex', 'embarked'], axis = 1 , inplace = True)
print(ndf.head())
print('\n')

# 4단계. 정규화
# survived  pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S
# survived (종속변수 y)
# pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S (독립변수 X)

# 4.1 독립변수와 종속변수(라벨)를 지정한다.
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S', 'child_women']] # 독립변수
y = ndf['survived'] # 종속변수


# 4.2 독립변수들을 정규화(normalization) 한다.
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
print(X)
print('\n')

# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다.
result = []
for i in range(10,20,1):
    
    from sklearn.model_selection import train_test_split
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = i)
    
    # 설명 : test_size = 0.3 에 의해서 7:3 비율로 훈련과 테스트를 나누고 
    # random_state = 10 에 의해서 나중에 split할 때도 항상 일정하게 split 할 수 있게 한다.
    
    
    # 6단계. 머신러닝 모델을 생성한다.(나이브 베이즈 분류모형을 사용)

    for j in range(1,10):
        
        from sklearn import tree
        tree_model = tree.DecisionTreeClassifier(criterion = 'entropy', max_depth = j)
        tree_model.fit(X_train, y_train)
        
        # 7단계. 테스트 데이터로 예측하기
        
        y_hat = tree_model.predict(X_test)
        
        # 8단계. 모형의 예측 능력을 평가한다. (confusion matrix 만들기!)
        from sklearn import metrics
        tree_matrix = metrics.confusion_matrix(y_test,y_hat)
    
    
        
        # 정확도 : (109 + 65) / (109 + 16 + 25 + 65)
        # 정확도 100%는 현실적으로 나오기 힘드니 (예측: 생존 , 실제: 사망 데이터인 25를 0으로 만드는 것을 궁극적으로 추구한다.)
        
        # confusion matrix에 대한 변수 지정 (tn,fp,fn,tp)
        tn, fp, fn, tp = metrics.confusion_matrix( y_test, y_hat ).ravel()
    
        
        f1_report = metrics.classification_report(y_test, y_hat)
    
        
        # 9단계 정확도 확인
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score( y_test, y_hat)
        result.append(accuracy)
        
        print('random : ', ',', i, 'mex_depth : ', j, ',', accuracy)

print('최대정확도는 : ', max(result))
print(result.index(max(result)), '번째순서')

## 반복문 x, 랜덤포레스트 코드 (submission.csv 제출용)

In [5]:
# 나이의 결측치를 바꾼 최빈값.
# Full code.

import seaborn as sns
import pandas as pd

# 1단계. csv --> 데이터 프레임으로 변환
df = sns.load_dataset('titanic')

# 컬럼이 모두다 출력될 수 있도록 출력할 열의 개수 한도를 늘리기
pd.set_option('display.max_columns', 15)
# print(df.head(), '\n')

mask1 = (df['age'] < 10) | (df['sex'] == 'female')
df['child_women'] = mask1.astype(int)

# 2.1 결측치를 확인하고 제거하거나 치환한다.
# 타이타닉 데이터 프레임의 자료형을 확인한다.

# print(df.info()) # 자료형 확인하는 방법
# print('\n')

# 2.2 결측치(NaN)을 확인한다.
print(df.isnull().sum(axis=0), '\n')

# 설명 : deck 결측치가 많아서 컬럼을 삭제해야함
# embark 와 embark_town이 같은 데이터(중복데이터)여서 컬럼을 하나 삭제해야함.
# embark 컬럼은 삭제한다.

# 2.3 deck 컬럼과 embark_town 컬럼을 삭제한다.
rdf = df.drop(['deck', 'embark_town'], axis =1)
# print(rdf.columns.values, '\n')

# 2.4 age(나이) 열에 나이가 없는 모든 행을 삭제한다.
# age 열에 891개의 행중에 177개가 NaN값

age_freq = rdf['age'].value_counts(dropna=True).idxmax()
print(age_freq)
print('\n')

print(df.isnull().sum(axis=0))
print('\n')

rdf['age'].fillna(age_freq, inplace = True)

print(len(rdf), '\n')
print(rdf.isnull().sum(axis=0))
print('\n')


# 2.5 embark 열의 NaN값을 승선도시중 가장 많이 출현한 값으로 치환하기.

most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
rdf['embarked'].fillna(most_freq, inplace = True)
print(rdf.isnull().sum(axis=0), '\n')

# 3. 범주형 데이터를 숫자형으로 변환하기

# 3.1 feature selection (분석에 필요한 속성을 선택)
ndf = rdf[['survived','pclass','sex','age','sibsp','parch','embarked', 'child_women']]
# print(ndf.head(), '\n')
# 선택된 컬럼중 2개(sex, embarked)가 범주형(문자형)이다.

# 3.2 범주형 데이터를 숫자로 변환(원핫 인코딩)
gender = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf,gender], axis = 1)

onehot_embarked = pd.get_dummies(ndf['embarked'], prefix = 'town') # 접두사를 정함.
ndf = pd.concat([ndf, onehot_embarked], axis = 1)

ndf.drop(['sex', 'embarked'], axis = 1 , inplace = True)
# print(ndf.head())
# print('\n')

# 4단계. 정규화
# survived  pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S
# survived (종속변수 y)
# pclass   age  sibsp  parch  female  male  town_C  town_Q  town_S (독립변수 X)

# 4.1 독립변수와 종속변수(라벨)를 지정한다.
X = ndf[['pclass','age','sibsp','parch','female','male','town_C','town_Q','town_S', 'child_women']] # 독립변수
y = ndf['survived'] # 종속변수

# 4.2 독립변수들을 정규화(normalization) 한다.
from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)


# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 19)

# 설명 : test_size = 0.3 에 의해서 7:3 비율로 훈련과 테스트를 나누고 
# random_state = 10 에 의해서 나중에 split할 때도 항상 일정하게 split 할 수 있게 한다.


# 6단계. 머신러닝 모델을 생성한다.(랜덤 포레스트를 사용)

from sklearn.ensemble import RandomForestClassifier

tree_model = RandomForestClassifier(n_estimators = 100, oob_score = True, random_state = 9)


tree_model.fit(X_train, y_train)

# oob 평가
"""
print(tree_model.oob_score_)
print('\n')
"""

# 7단계. 테스트 데이터로 예측하기

y_hat = tree_model.predict(X_test)


# 8단계. 모형의 예측 능력을 평가한다. (confusion matrix 만들기!)
from sklearn import metrics
tree_matrix = metrics.confusion_matrix(y_test,y_hat)

# confusion matrix에 대한 변수 지정 (tn,fp,fn,tp)
tn, fp, fn, tp = metrics.confusion_matrix( y_test, y_hat ).ravel()


f1_report = metrics.classification_report(y_test, y_hat)
print(f1_report,'\n')

# 9단계 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
print(accuracy)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
child_women    891 non-null int32
dtypes: float64(2), int32(1), int64(5), object(5)
memory usage: 87.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 12 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 

## 반복문 o, 랜덤포레스트 코드 

In [ ]:
# 랜덤 포레스트 적용
# age 부분 최빈값으로 대체
# fare 부분 이상치 제거 및 최빈값으로 NaN 대체
# deck 부분 컬럼 삭제


import pandas as pd


df = pd.read_csv("d:\\data\\tat\\train.csv")
df2 = pd.read_csv("d:\\data\\tat\\test.csv")

pd.set_option('display.max_columns', 15)


mask1 = (df['Age'] < 10) | (df['Sex'] == 'female')
mask2 = (df2['Age'] < 10) | (df2['Sex'] == 'female')

df['child_women'] = mask1.astype(int)
df2['child_women'] = mask2.astype(int)


print(df.info())
print(df2.info())
print('\n')

print(df.isnull().sum(axis=0), '\n')
print(df2.isnull().sum(axis=0), '\n')

# Cabin 칼럼 날려버리기
rdf = df.drop(['Cabin'], axis =1)
rdf2 = df2.drop(['Cabin'], axis =1)

print(rdf.columns.values, '\n')
print(rdf2.columns.values, '\n')


# 나이값 Age컬럼의 결측치를 최빈값으로 대체
age_freq1 = rdf['Age'].value_counts(dropna=True).idxmax()
age_freq2 = rdf2['Age'].value_counts(dropna=True).idxmax()

print(age_freq1) # 24.0
print(age_freq2) # 24.0

rdf['Age'].fillna(age_freq1, inplace = True)
rdf2['Age'].fillna(age_freq2, inplace = True)


# Embarked 결측치 값을 최빈값으로 대체
# test.csv 에는 Embarked에 결측치 값이 없으므로 과정 생략.
most_freq1 = rdf['Embarked'].value_counts(dropna=True).idxmax()
# most_freq2 = rdf2['Embarked'].value_counts(dropna=True).idxmax()

rdf['Embarked'].fillna(most_freq1, inplace = True)
# rdf2['Embarked'].fillna(most_freq2, inplace = True)

print(rdf.isnull().sum(axis=0), '\n')
print(rdf2.isnull().sum(axis=0), '\n')




ndf = rdf[['Survived','Pclass','Sex','Age','SibSp','Parch','Embarked', 'child_women', 'Fare']]
ndf2 = rdf2[['Pclass','Sex','Age','SibSp','Parch','Embarked', 'child_women', 'Fare']]

print(ndf.head(), '\n')
print(ndf2.head(), '\n')

gender = pd.get_dummies(ndf['Sex'])
gender2 = pd.get_dummies(ndf2['Sex'])

ndf = pd.concat([ndf,gender], axis = 1)
ndf2 = pd.concat([ndf2,gender2], axis = 1)

onehot_embarked = pd.get_dummies(ndf['Embarked'], prefix = 'town') # 접두사를 정함.
onehot_embarked2 = pd.get_dummies(ndf2['Embarked'], prefix = 'town') # 접두사를 정함.

ndf = pd.concat([ndf, onehot_embarked], axis = 1)
ndf2 = pd.concat([ndf2, onehot_embarked2], axis = 1)

ndf.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)
ndf2.drop(['Sex', 'Embarked'], axis = 1 , inplace = True)

# 운임값(fare)의 이상치를 제거 (훈련데이터만)
local_std1 = ndf['Fare'].std() * 5


result1 = ndf['Fare'][rdf['Fare'] > local_std1]
print(result1)


ndf = ndf[:][rdf['Fare'] < local_std1]


print(ndf.isnull().sum(axis=0), '\n')
print(ndf2.isnull().sum(axis=0), '\n')


# 운임값(fare)의 결측치를 최빈값으로 채움.
fare_freq1 = ndf['Fare'].value_counts(dropna=True).idxmax()
fare_freq2 = ndf2['Fare'].value_counts(dropna=True).idxmax()


ndf['Fare'].fillna(fare_freq1, inplace = True)
ndf2['Fare'].fillna(fare_freq2, inplace = True)

print(ndf.isnull().sum(axis=0), '\n')
print(ndf2.isnull().sum(axis=0), '\n')


print(ndf.head())
print('\n')

print(ndf2.head())
print('\n')

X = ndf[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S', 'child_women', 'Fare']] # 독립변수
y = ndf['Survived'] # 종속변수

test = ndf2[['Pclass','Age','SibSp','Parch','female','male','town_C','town_Q','town_S', 'child_women', 'Fare']]

from sklearn import preprocessing

X = preprocessing.StandardScaler().fit(X).transform(X)
test = preprocessing.StandardScaler().fit(test).transform(test)

print(X)
print(test)


# 5단계 . 데이터셋을 훈련데이터와 테스트 데이터로 나눈다.

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 14)

# 설명 : test_size = 0.3 에 의해서 7:3 비율로 훈련과 테스트를 나누고 
# random_state = 10 에 의해서 나중에 split할 때도 항상 일정하게 split 할 수 있게 한다.


# 6단계. 머신러닝 모델을 생성한다.(랜덤 포레스트를 사용)

from sklearn.ensemble import RandomForestClassifier

tree_model = RandomForestClassifier(n_estimators = 60, oob_score = True, random_state = 15)


tree_model.fit(X_train, y_train)

# oob 평가
"""
print(tree_model.oob_score_)
print('\n')
"""

y_hat = tree_model.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])


# confusion matrix에 대한 변수 지정 (tn,fp,fn,tp)
from sklearn import metrics
tn, fp, fn, tp = metrics.confusion_matrix( y_test, y_hat ).ravel()
print(tn,fp,fn,tp)
print('\n')


# 9단계 정확도 확인
from sklearn.metrics import accuracy_score
accuracy = accuracy_score( y_test, y_hat)
print(accuracy)


# Prediction
y_pred = tree_model.predict(test)

sample_submission=pd.read_csv('d:\\data\\tat\\gender_submission.csv', index_col=0)


# Submission
submission = pd.DataFrame(data=y_pred,columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('d:\\data\\tat\\submission_randomforest.csv', index=True)